<a href="https://colab.research.google.com/github/dguti97/Taller2_ANN/blob/master/Taller2_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# NUEVO INTENTO

In [ ]:
import pandas as pd

df=pd.read_csv("train_source_tweets.txt",delimiter="\t",names=['id', 'text'])
print(df)
label=pd.read_csv("train_labels.csv")
print(label)

                      id                                               text
0     692735698349199360  north korea 'planning some kind of rocket laun...
1     525008463819464704  meet kevin vickers, the hero who shot down the...
2     505611045897924608  15 year old who "swatted" gamer convicted of d...
3     693466724822323200  audio recordings reveal cpr started 11 minutes...
4     510922415468449792  awful mt @scclemons:uk aid worker david haines...
...                  ...                                                ...
1921  515598200796090369  #workplace hero shoots #altonnolen #stabbings ...
1922  535257207991205888  russian boy shocked by electricity now claims ...
1923  524944399890124801  recap: gunman shot dead inside parliament buil...
1924  519965814175567872  a vandal painted a brilliantly simple penis on...
1925  523939598691741696  cool dog! mt @cbsnews: goliath encounter: pupp...

[1926 rows x 2 columns]
           label                  id
0      non-rumor  69273569

In [ ]:
df = df.sort_values(by=['id'])
label = label.sort_values(by=['id'])

In [ ]:
df['text']

1339    seriously? racist mcdonald’s sign is obviously...
725     hoax: mcdonald's issues official statement on ...
1725    #psa please do not drink any pepsi soda, a wor...
1675    42 million dead in bloodiest black friday week...
941     42 million dead in bloodiest black friday week...
                              ...                        
58      black men have legitimate reason to run from p...
1432    a black man who runs from police merely might ...
663     this network of tunnels is from the stone age ...
1624    well, this is a new one: chelsea clinton impli...
937     chelsea clinton implies here that marijuana ca...
Name: text, Length: 1926, dtype: object

In [ ]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

In [ ]:
df['text']

0       seriously racist mcdonalds sign obviously hoa url
1       hoa mcdonalds issues official statement racist...
2       #psa please drink pepsi soda worker company pu...
3        million dead bloodiest black friday weekend r...
4        million dead bloodiest black friday weekend r...
                              ...                        
1921    black men legitimate reason run police #massac...
1922    black man runs police merely might trying avoi...
1923    network tunnels stone age however one knows co...
1924    well new one chelsea clinton implies marijuana...
1925       chelsea clinton implies marijuana kill uhh url
Name: text, Length: 1926, dtype: object

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5550
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 16
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
MAX_NB_WORDS = len(word_index)+1
print('Found %s unique tokens.' % len(word_index))

Found 5515 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (1926, 100)


In [ ]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  744,  745,  746,  931, 2507,  433,
          1], dtype=int32)

In [ ]:
Y = pd.get_dummies(label['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (1926, 4)


In [ ]:
label_name=['false','non-rumor','true','unverified']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1733, 100) (1733, 4)
(193, 100) (193, 4)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(25, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
batch_size = 32 

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 1559 samples, validate on 174 samples
Epoch 1/15
1559/1559 [==============================] - 3s 2ms/step - loss: 1.3801 - accuracy: 0.2765 - val_loss: 1.3719 - val_accuracy: 0.2931
Epoch 2/15
1559/1559 [==============================] - 3s 2ms/step - loss: 1.3540 - accuracy: 0.3438 - val_loss: 1.3495 - val_accuracy: 0.3736
Epoch 3/15
1559/1559 [==============================] - 2s 2ms/step - loss: 1.2450 - accuracy: 0.5061 - val_loss: 1.1964 - val_accuracy: 0.5230
Epoch 4/15
1559/1559 [==============================] - 2s 2ms/step - loss: 0.9407 - accuracy: 0.7325 - val_loss: 0.9578 - val_accuracy: 0.6494
Epoch 5/15
1559/1559 [==============================] - 2s 2ms/step - loss: 0.5968 - accuracy: 0.8538 - val_loss: 0.8128 - val_accuracy: 0.7069
Epoch 6/15
1559/1559 [==============================] - 3s 2ms/step - loss: 0.3669 - accuracy: 0.9230 - val_loss: 0.7690 - val_accuracy: 0.7126
Epoch 7/15
1559/1559 [==============================] - 2s 2ms/step - loss: 0.2645 - accu

In [ ]:
df2=pd.read_csv("test_source_tweets.txt",delimiter="\t",names=['id', 'text'])
df2=df2.sort_values(by=['id'])

In [ ]:
df2['text']

196    deep-fried left wings demo-crab cakes barack-a...
61     a photo of black nurses saving the life of a k...
136    obama files federal charges against george zim...
57     revealed: #fukushima leaking radioactive water...
110    details: radioactive water from under #fukushi...
                             ...                        
133    wow. anthony johnson's knockout win is the 3rd...
111    powell breaks silence: clintons trying to 'pin...
64     our quote of the day is from american writer d...
169    massachusetts court finds that it’s ‘reasonabl...
43     black men may have cause to run from police, m...
Name: text, Length: 213, dtype: object

In [ ]:
df2['text'] = df2['text'].apply(clean_text)
df2['text'] = df2['text'].str.replace('\d+', '')
df2['text']

196    deepfried left wings democrab cakes barackamol...
61         photo black nurses saving life kkk member url
136    obama files federal charges george zimmerman f...
57     revealed #fukushima leaking radioactive water ...
110    details radioactive water #fukushima risen cm ...
                             ...                        
133    wow anthony johnsons knockout win rdquickest f...
111    powell breaks silence clintons trying pin scan...
64          quote day american writer dorothy parker url
169    massachusetts court finds reasonable black men...
43     black men may cause run police massachusetts h...
Name: text, Length: 213, dtype: object

In [ ]:
X = tokenizer.texts_to_sequences(df2['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (213, 100)


In [ ]:
prediction = model.predict_classes(X)

In [ ]:
prediction

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0,
       1, 3, 3, 3, 3, 3, 3, 3, 2, 0, 0, 2, 2, 2, 2, 1, 3, 3, 2, 0, 1, 1,
       0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 3, 3, 0, 0, 0, 3, 0, 0, 3, 2, 2, 2, 2, 3, 1, 2, 2, 3, 2,
       2, 2, 2, 3, 3, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 3, 0, 0, 1, 1, 1, 2,
       3, 0, 0, 3, 0, 0, 0, 3, 1, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 1,
       3, 1, 3, 3, 3, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3])

In [ ]:
prediction_label = list()
for i in range(prediction.shape[0]):
    prediction_label.append(label_name[prediction[i]])
prediction_label = np.array(prediction_label)

In [ ]:
submit = pd.DataFrame({"label": prediction_label, "id": df2['id'].values})

In [ ]:
submit.to_csv('submission_DevelopIO6.csv',index=False)

Otra opcion de predecir

In [ ]:
pred = model.predict(X)

In [ ]:
for i in pred:
  print(i)
  print(label_name[np.argmax(i)])

[9.9999273e-01 2.3792045e-06 5.3895980e-07 4.2778902e-06]
false
[0.10553202 0.36824456 0.36797473 0.15824868]
non-rumor
[0.6248569  0.00840126 0.35930482 0.00743703]
false
[0.06455726 0.2887859  0.11936095 0.5272959 ]
unverified
[0.3408893  0.12364438 0.00508037 0.530386  ]
unverified
[0.5613935  0.20385285 0.07072824 0.16402541]
false
[0.4899445  0.11174774 0.31360653 0.08470125]
false
[0.18681595 0.03068253 0.7392534  0.04324804]
true
[0.01722401 0.02866641 0.93074954 0.02336005]
true
[9.7096269e-04 1.1722147e-03 9.9683207e-01 1.0248586e-03]
true
[0.10781299 0.08269083 0.70340586 0.10609028]
true
[9.8316944e-01 8.5700788e-03 6.6732190e-04 7.5931321e-03]
false
[0.00687364 0.00456395 0.97404736 0.01451506]
true
[0.8337643  0.09850843 0.04583997 0.02188725]
false
[0.39878687 0.4987059  0.00575547 0.09675179]
non-rumor
[0.00287605 0.00748504 0.98753256 0.00210644]
true
[0.0357843  0.15587032 0.7760746  0.03227078]
true
[0.18948606 0.712372   0.0573806  0.04076133]
non-rumor
[0.00462242 0